In [20]:
import pandas as pd
import numpy as np
import psycopg2
from sqlalchemy import create_engine
from itertools import chain
from itertools import combinations
from scipy.sparse import csr_matrix

In [3]:
product_list = pd.read_excel('shipment_linear.xlsx', engine = 'openpyxl')
product_codes = product_list['Code'].tolist()

In [37]:
db_config = {'user': 'XXXXXX','password': 'XXXXXX','host': 'XXXXXX','port': '5432', 'database': 'XXXXXX'}
engine = create_engine(f"postgresql+psycopg2://{db_config['user']}:{db_config['password']}@{db_config['host']}:{db_config['port']}/{db_config['database']}")
query = f"""
    SELECT imtrn.xitem, imtrn.xyear,imtrn.xdocnum, imtrn.xqty
    FROM imtrn 
    JOIN caitem
    ON imtrn.xitem = caitem.xitem
    JOIN opddt
    ON imtrn.xdocnum = opddt.xdornum AND imtrn.xitem = opddt.xcode
    WHERE imtrn.zid = 100001
    AND caitem.zid = 100001
    AND opddt.zid = 100001
    """
sales_data = pd.read_sql(query, con=engine)
sales_data = sales_data[sales_data['xyear']>=2023]

In [38]:
grouped_data = sales_data.groupby('xdocnum')['xitem'].apply(list).reset_index()
grouped_data.columns = ['InvoiceID', 'Items']

In [39]:
grouped_data

,InvoiceID,Items
0,DO--155128,[1780]
1,DO--155139,[1675]
2,DO--155140,"[1714, 1131, 1522, 1576, 1419, 1528, 1624]"
3,DO--155141,[1744]
4,DO--155142,[1455]
...,...,...
320953,DO--476241,"[1185, 16081, 16102, 12933]"
320954,DO--476242,"[2110, 1726]"
320955,DO--476243,[14875]
320956,DO--476244,[1851]


In [40]:
all_items = list(chain(*grouped_data['Items']))
unique_items = list(set(all_items))

In [41]:
basket_matrix = pd.DataFrame(
    [[item in items for item in unique_items] for items in grouped_data['Items']],
    columns=unique_items
)

# item_to_idx = {item: idx for idx, item in enumerate(unique_items)}

# row_indices = []
# col_indices = []

# for idx, items in enumerate(grouped_data['Items']):
#     for item in items:
#         row_indices.append(idx)
#         col_indices.append(item_to_idx[item])

# # Using sparse matrix
# basket_sparse = csr_matrix((np.ones(len(row_indices)), (row_indices, col_indices)), shape=(len(grouped_data), len(unique_items)))

In [42]:
support_item = basket_matrix.mean()

In [43]:
pair_support = {}
for item1, item2 in combinations(unique_items, 2):
    pair_support[(item1, item2)] = basket_matrix[item1].mul(basket_matrix[item2]).mean()
    

# pair_support = {}

# for item1, item2 in combinations(range(len(unique_items)), 2):
#     dot product
#     support_count = basket_sparse[:, item1].multiply(basket_sparse[:, item2]).sum()
#     pair_support[(unique_items[item1], unique_items[item2])] = support_count / basket_sparse.shape[0]

In [44]:
rules = []

for (item1, item2), support_pair in pair_support.items():
    if support_pair > 0:

        confidence_item1_to_item2 = support_pair / support_item[item1]
        lift_item1_to_item2 = confidence_item1_to_item2 / support_item[item2]

        rules.append({
            'Antecedent': item1,
            'Consequent': item2,
            'Support': support_pair,
            'Confidence': confidence_item1_to_item2,
            'Lift': lift_item1_to_item2
        })

# Convert rules list to DataFrame
rules_df = pd.DataFrame(rules)

In [27]:
# rules_df.to_excel('item_basket.xlsx')

In [49]:
#Find the mean number of products bought per invoice
grouped_data['ItemCount'] = grouped_data['Items'].apply(len)
mean_items_per_invoice = round(grouped_data['ItemCount'].mean())

In [50]:
top_4_per_product = rules_df.sort_values(by=['Antecedent', 'Lift'], ascending=[True, False]).groupby('Antecedent').head(mean_items_per_invoice)

In [55]:
top_4_per_product = top_4_per_product[top_4_per_product['Antecedent'].isin(product_codes)]

In [56]:
results = []

for _, row in top_4_per_product.iterrows():
    antecedent = row['Antecedent']
    consequent = row['Consequent']
    
    # Filter transactions where both antecedent and consequent are present
    joint_transactions = sales_data[sales_data['xitem'].isin([antecedent, consequent])]
    joint_invoice_ids = joint_transactions.groupby('xdocnum').filter(lambda x: set([antecedent, consequent]).issubset(set(x['xitem']))).xdocnum.unique()
    
    joint_sales = sales_data[sales_data['xdocnum'].isin(joint_invoice_ids)]
    
    antecedent_qty = joint_sales[joint_sales['xitem'] == antecedent]['xqty'].sum()
    consequent_qty = joint_sales[joint_sales['xitem'] == consequent]['xqty'].sum()
    sales_ratio = consequent_qty / antecedent_qty if antecedent_qty > 0 else None
    
    # Append results
    results.append({
        'Antecedent': antecedent,
        'Consequent': consequent,
        'JointAntecedentQty': antecedent_qty,
        'JointConsequentQty': consequent_qty,
        'SalesRatio': sales_ratio
    })

joint_sales_ratio_df = pd.DataFrame(results)


In [68]:
joint_sales_ratio_df[joint_sales_ratio_df['Antecedent']=='0113']

,Antecedent,Consequent,JointAntecedentQty,JointConsequentQty,SalesRatio
15,0113,1598,0.333,1.0,3.003003
16,0113,0555,8.000,30.0,3.750000
17,0113,1802,1.080,3.0,2.777778


In [63]:
top_4_per_product = top_4_per_product.merge(joint_sales_ratio_df, on=['Antecedent','Consequent'], how = 'left')


In [65]:
top_4_per_product = top_4_per_product.drop(columns = ['JointAntecedentQty_x','JointAntecedentQty_y','JointConsequentQty_x', 'JointConsequentQty_y','SalesRatio_x','SalesRatio_y'], axis = 1)

In [67]:
top_4_per_product.to_excel('final_basket.xlsx')

In [ ]:
# how to optimize linear optimization with basket constraint